In [ ]:
import json
  
# Opening JSON file
file = open('/content/drive/MyDrive/NatureLanguageProgram/training_data/translation2019zh_valid.json')
txt_english = []
txt_chinese = []
txt = []

for line in file.readlines():
  data = json.loads(line)
  txt_english.append(data['english'])
  txt_chinese.append(data['chinese'])
  txt.append(data)

# Iterating through the json
# list
for i in txt:
  print(i)
print(type(txt))

串流輸出內容已截斷至最後 5000 行。
{'english': 'A four-wheeled carriage, often with a hood that folds back and having a handle for pushing, used for wheeling an infant about.', 'chinese': '一种四轮车，经常带有一个向后折叠的车篷和一个用来推车的把手，用来推着婴儿到各处走动。'}
{'english': "Climate change, earthquakes and volcanic eruptions result from stresses on the Earth's crust during the shift.", 'chinese': '气候变化、地震和火山喷发都是地壳在运动时产生的内力所造成的。'}
{'english': "I took leave a teacher's wife to go to a school, continue of have a class.", 'chinese': '我告别了师母就去了学校，继续的上课。'}
{'english': 'The writers draw a conclusion that the feasible planting grass for southern airport are Bermudagrass, Zoysiagrass and Bahiagrass, and the feasible planting method is liqidpress spray technology.', 'chinese': '实践证明，适宜南方机场飞行区种植的草坪草种为狗牙根、结缕草和百喜草等，适宜的建植方法为液压喷播技术。'}
{'english': 'Oil exploration around the Falklands has angered Argentina which challenges British sovereignty over the islands it calls the Malvinas.', 'chinese': '福克兰群岛的石油勘探激怒了阿根廷，他们一直在这个自称为马尔维纳斯群岛的地方挑战英国主权。'}
{

In [11]:
import os
import time
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import clear_output

